# Z5D Validation Analysis — N₁₂₇\n\nThis notebook analyzes the dataset produced by `experiments/z5d_validation_n127.py` and visualizes whether high Z5D alignment (more negative `z5d_score`, i.e. higher `z5d_resonance = -z5d_score`) enriches candidates near the true factors of **N₁₂₇**.\n\n**Important:** factorization candidates here are ~1e19 and exceed signed 64-bit range. Avoid `int64/uint64` conversions; treat large integers as strings or Python `int`.

In [ ]:
from pathlib import Path\nimport json\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom matplotlib.colors import LogNorm\n\nSUMMARY_PATH = Path("../artifacts/z5d_validation_n127_summary.json")\nCSV_PATH = Path("../artifacts/z5d_validation_n127.csv")\n\nsummary = json.loads(SUMMARY_PATH.read_text())\nsqrt_n = int(summary["constants"]["sqrt_n"])\np = int(summary["constants"]["p"])\nq = int(summary["constants"]["q"])\n\nsummary["stats"]

In [ ]:
# Helper: load only the columns we need, keeping large integers as strings.\nusecols = ["candidate", "z5d_score", "z5d_resonance", "nearest_factor_dist"]\ndtype = {"candidate": "string", "nearest_factor_dist": "string"}\n\ndf = pd.read_csv(CSV_PATH, usecols=usecols, dtype=dtype)\n\n# For plotting, float64 is fine (we only need approximate positions).\ndf["candidate_f"] = df["candidate"].astype("float64")\ndf["signed_distance_from_sqrt"] = df["candidate_f"] - float(sqrt_n)\ndf["nearest_factor_dist_f"] = df["nearest_factor_dist"].astype("float64")\n\ndf.head()

In [ ]:
# 2D density: signed distance from sqrt(N) vs z5d_score\nplt.figure(figsize=(10, 5))\nplt.hist2d(\n    df["signed_distance_from_sqrt"],\n    df["z5d_score"],\n    bins=250,\n    norm=LogNorm(),\n    cmap="viridis",\n)\nplt.colorbar(label="count")\nplt.xlabel("candidate - sqrt(N)")\nplt.ylabel("z5d_score (lower is better)")\nplt.title("Z5D score vs signed distance from sqrt(N_127)")\nplt.tight_layout()\nplt.show()

In [ ]:
# Scatter (downsample) : z5d_score vs distance to nearest true factor\nsample = df.sample(n=min(50_000, len(df)), random_state=127)\n\nplt.figure(figsize=(10, 5))\nplt.scatter(sample["nearest_factor_dist_f"], sample["z5d_score"], s=2, alpha=0.15)\nplt.xscale("log")\nplt.xlabel("min(|c-p|, |c-q|)")\nplt.ylabel("z5d_score")\nplt.title("Z5D score vs distance to nearest true factor")\nplt.tight_layout()\nplt.show()